In [13]:
import os

In [14]:
%pwd

'D:\\KidneyClassification-MLFlow'

In [4]:
os.chdir("../")

In [5]:
%pwd

'D:\\KidneyClassification-MLFlow'

In [16]:
import dagshub
dagshub.init(repo_owner='swathy', repo_name='KidneyClassification-MLFlow', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

C:\Users\rampr\anaconda3\envs\kidney\lib\site-packages\rich\live.py:260: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=21859277-dda1-4e80-b410-122473ce1345&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=25135867f2788a0670a94e132292b12a580c4bd5dd32dbdfcda67000d4ee241d


[2026-01-28 19:18:19,218: INFO: _client: HTTP Request: POST https://dagshub.com/login/oauth/middleman "HTTP/1.1 200 OK"]


[2026-01-28 19:18:20,925: INFO: _client: HTTP Request: POST https://dagshub.com/login/oauth/access_token "HTTP/1.1 200 OK"]
[2026-01-28 19:18:22,455: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Accessing as swathy

[2026-01-28 19:18:22,489: INFO: helpers: Accessing as swathy]
[2026-01-28 19:18:23,670: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/swathy/KidneyClassification-MLFlow "HTTP/1.1 200 OK"]
[2026-01-28 19:18:24,897: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "swathy/KidneyClassification-MLFlow"

[2026-01-28 19:18:24,918: INFO: helpers: Initialized MLflow to track repo "swathy/KidneyClassification-MLFlow"]


Repository swathy/KidneyClassification-MLFlow initialized!

[2026-01-28 19:18:24,927: INFO: helpers: Repository swathy/KidneyClassification-MLFlow initialized!]
🏃 View run handsome-bird-389 at: https://dagshub.com/swathy/KidneyClassification-MLFlow.mlflow/#/experiments/0/runs/ddce4da21a8f4613a0b2641d9479ed23
🧪 View experiment at: https://dagshub.com/swathy/KidneyClassification-MLFlow.mlflow/#/experiments/0


In [23]:

model = tf.keras.models.load_model("artifacts/training/model.h5")

[2026-01-28 19:25:26,080: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]


In [24]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [26]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [27]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/entbappy/Kidney-Disease-Classification-MLflow-DVC.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [28]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [29]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [30]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2026-01-28 19:26:50,224: INFO: common: yaml file: config\config.yaml loaded successfully]
[2026-01-28 19:26:50,269: INFO: common: yaml file: params.yaml loaded successfully]
[2026-01-28 19:26:50,281: INFO: common: created directory at: artifacts]
[2026-01-28 19:26:50,702: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 139 images belonging to 2 classes.
9/9 ━━━━━━━━━━━━━━━━━━━━ 47s 5s/step - accuracy: 0.5180 - loss: 6.2310
[2026-01-28 19:27:38,456: INFO: common: json file saved at: scores.json]


2026/01/28 19:27:40 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/01/28 19:27:42 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.


🏃 View run upbeat-carp-758 at: https://dagshub.com/swathy/KidneyClassification-MLFlow.mlflow/#/experiments/0/runs/3d91d2077430484fb3b8bb2c4559d026
🧪 View experiment at: https://dagshub.com/swathy/KidneyClassification-MLFlow.mlflow/#/experiments/0


MlflowException: API request to endpoint /api/2.0/mlflow/registered-models/create failed with error code 403 != 200. Response body: ''